## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [1]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2022-03-24 09:57:05--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/obaitrix9jyu84r/quora.txt [following]
--2022-03-24 09:57:36--  https://www.dropbox.com/s/dl/obaitrix9jyu84r/quora.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc37aa2f8a071d00f1128dc71b14.dl.dropboxusercontent.com/cd/0/get/BiBT9HYGp6zDSebDn0rHrcSSCIB_UUln2kfNcrq-JSYoxf-QXk5VY5cT13Iz6K1fkovo85DsovCE9MIWJXUq_U34-IzNNO1ZMNrSYrGmYLZbAtbW_QEoYVCCdejgNQqU6S9ykQtGzo1r_u020cpcHOkhQEqO93jGoh2XAf71NdOYvw/file?dl=1# [following]
--2022-03-24 09:57:38--  https://uc37aa2f8a071d00f1128dc71b14.dl.dropboxusercontent.com/cd/0/get/BiBT9HYGp6zDSebDn0rHrcSSCIB_UUln2kfNcrq-JSYoxf-QXk5VY5cT13Iz6K1fkovo85Ds

In [14]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))

__Tokenization:__ a typical first step for an nlp task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [2]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [6]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(x.lower()) for x in data]


In [7]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [8]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


In [12]:
data_tok

[['can',
  'i',
  'get',
  'back',
  'with',
  'my',
  'ex',
  'even',
  'though',
  'she',
  'is',
  'pregnant',
  'with',
  'another',
  'guy',
  "'",
  's',
  'baby',
  '?'],
 ['what',
  'are',
  'some',
  'ways',
  'to',
  'overcome',
  'a',
  'fast',
  'food',
  'addiction',
  '?'],
 ['who',
  'were',
  'the',
  'great',
  'chinese',
  'soldiers',
  'and',
  'leaders',
  'who',
  'fought',
  'in',
  'ww2',
  '?'],
 ['what', 'are', 'zip', 'codes', 'in', 'the', 'bay', 'area', '?'],
 ['why',
  'was',
  'george',
  'rr',
  'martin',
  'critical',
  'of',
  'jk',
  'rowling',
  'after',
  'losing',
  'the',
  'hugo',
  'award',
  '?'],
 ['what', 'can', 'i', 'do', 'to', 'improve', 'my', 'immune', 'system', '?'],
 ['how',
  'is',
  'your',
  'relationship',
  'with',
  'your',
  'mother',
  'in',
  'law',
  '?'],
 ['how',
  'does',
  'one',
  'get',
  'free',
  'psn',
  'codes',
  '/',
  'vita',
  'codes',
  '?'],
 ['what', 'is', 'your', 'review', 'of', 'osquery', '?'],
 ['how', 'can', '

__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [6]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

In [7]:
# now you can get word vectors !
model.get_vector('anything')

array([-3.2546203 , -0.2428707 ,  0.64118904,  0.7614322 ,  1.356812  ,
        1.5289421 ,  0.8279628 , -4.423762  ,  1.3965129 ,  2.8556583 ,
       -0.70461905,  3.415799  ,  4.4670258 ,  0.8396054 ,  1.717268  ,
       -2.2519815 ,  0.57377756, -1.0630941 ,  0.5445976 , -3.3640559 ,
       -1.0340649 , -1.4066734 , -0.74209434, -3.7282908 ,  1.7862296 ,
       -2.4778566 , -0.27434057, -0.0193115 , -0.45847988, -0.95869464,
       -1.6496156 , -0.4966366 ], dtype=float32)

In [8]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.955434262752533),
 ('chocolate', 0.9332507252693176),
 ('beans', 0.9276328086853027),
 ('fruit', 0.9274473190307617),
 ('pasta', 0.9252123236656189),
 ('banana', 0.924961268901825),
 ('butter', 0.9236361980438232),
 ('sauce', 0.9235565066337585),
 ('vodka', 0.9207842946052551),
 ('cheese', 0.9144188165664673)]

In [9]:
model.most_similar('monkey')

[('floyd', 0.8816367387771606),
 ('jungle', 0.8811105489730835),
 ('spirit', 0.8751728534698486),
 ('cage', 0.869387149810791),
 ('paradise', 0.8687356114387512),
 ('sith', 0.8614618182182312),
 ('ki', 0.8553956151008606),
 ('marilyn', 0.8530427813529968),
 ('goddess', 0.8455272912979126),
 ('hood', 0.8436485528945923)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [10]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[===-----------------------------------------------] 6.1% 23.4/387.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=======-------------------------------------------] 14.9% 57.8/387.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================-----------------] 67.4% 260.9/387.1MB downloaded

...

[=======================================-----------] 79.8% 308.8/387.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===========================================-------] 87.5% 338.8/387.1MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=================================================-] 99.3% 384.4/387.1MB downloaded


In [13]:
model.most_similar(positive=["coder", "money"], negative=["aweful"])

[('cash', 0.6015129685401917),
 ('business', 0.5539237856864929),
 ('medical', 0.5398741960525513),
 ('buy', 0.5316650867462158),
 ('free', 0.5308775901794434),
 ('pay', 0.5298088192939758),
 ('job', 0.5288501381874084),
 ('specialist', 0.5187565088272095),
 ('designer', 0.5182684659957886),
 ('jobs', 0.5167829990386963)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [39]:
words = sorted(model.key_to_index.keys(), 
               key=lambda word: model.key_to_index[word],
               reverse=True)[:5000]

print(words[::100])

['ﾟﾟﾟｵﾔｽﾐｰ', '자는것도', '민통이', '過去の貴重な事例に学ぶことが重要だと思いました', '未開封であれば約', '分からないな', 'ツイッターで一番稼げる方法', 'の人が挫折する障害を取り除いた即金ビジネスがあります', 'お夕飯', 'چیزای', 'وجعكِ', 'مايجيش', 'عالَم', 'تگلمت', 'الملائم', 'إنِيّ', 'пустыню', 'αρχίσουν', 'waddah', 'trenroca', 'suadades', 'sayeed', 'pv-gs', 'pagini', 'nsombong', 'mysound', 'macklemoore', 'kinahanglan', 'infiltrando', 'grinar', 'eşinizi', 'dialoguen', 'cnant', 'boiolice', 'askush', 'ﾖﾛｼｸお願いします', '응원하면', '디테일한', '許してはならない', '日間返金保障n', '円分又は', 'ヘイｐ', 'やばっw', 'さすがですｗ', 'เขาจะไม่พูดด้วยn', 'وفيته', 'مكانـه', 'فرصهم', 'حوبك', 'ببعضهما']


In [40]:
# for each word, compute it's vector with model
word_vectors = np.array([model.get_vector(word) for word in words])

In [41]:
word_vectors

array([[-0.028777 , -0.72607  , -0.8277   , ..., -0.69694  , -0.76517  ,
        -1.0901   ],
       [-0.38313  ,  0.064609 ,  0.27384  , ..., -0.16396  , -0.26971  ,
        -0.69941  ],
       [ 0.28892  ,  0.25407  ,  0.11113  , ..., -0.6158   ,  0.046846 ,
        -0.086387 ],
       ...,
       [ 0.57385  ,  0.13648  , -0.28442  , ...,  0.64121  ,  0.060009 ,
         0.46176  ],
       [-0.15065  ,  0.27267  , -0.52086  , ...,  0.12326  ,  0.0048568,
        -0.32654  ],
       [-0.48331  ,  0.045266 ,  0.095662 , ...,  0.48866  , -0.31262  ,
        -0.12948  ]], dtype=float32)

In [42]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is __P__rincipial __C__omponent __A__nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [43]:
from sklearn.decomposition import PCA

# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(2).fit_transform(word_vectors)

# Transforming the variables
from sklearn.preprocessing import StandardScaler
word_vectors_pca = StandardScaler().fit_transform(word_vectors_pca)
word_vectors_pca

array([[ 1.481882  , -0.6310089 ],
       [-1.1309417 , -2.4802547 ],
       [ 0.05666419, -1.498018  ],
       ...,
       [-0.74828315, -0.40449917],
       [-0.5541032 , -0.40947345],
       [-1.2941034 ,  0.14657937]], dtype=float32)

In [28]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [15]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [45]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

Figure(id='1365', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [47]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = TSNE(2).fit_transform(word_vectors)

word_tsne = StandardScaler().fit_transform(word_vectors_pca)


In [48]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1507', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [52]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    n = 0
    i = 0
    for word in tokenizer.tokenize(phrase.lower()):
        if word in model:
            vector += model.get_vector(word)
            i      += 1
    if i > 0:
        vector = vector / i
    return vector
        
    

In [53]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [58]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [59]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [60]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [61]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1660', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [62]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [63]:
from sklearn.metrics.pairwise import cosine_distances
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    query_emb = get_phrase_embedding(query)
    cos_dist = [cosine_distances([query_emb], [vect])[0][0] for vect in data_vectors]
    indexes = np.argsort(cos_dist)[:k]
    return np.array(data)[indexes]

In [72]:
query_emp = get_phrase_embedding("How do I get to the dark web?")
cos_dist = [cosine_distances([query_emp], [vect])[0][0] for vect in data_vectors]
indexes = np.argsort(cos_dist)[:20]

In [64]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [73]:
find_nearest(query="How does Trump?", k=10)

array(['What does Donald Trump think about Israel?\n',
       'What books does Donald Trump like?\n',
       'What does Donald Trump think of India?\n',
       'What does India think of Donald Trump?\n',
       'What does Donald Trump think of China?\n',
       'What does Donald Trump think about Pakistan?\n',
       'What companies does Donald Trump own?\n',
       'What does Dushka Zapata think about Donald Trump?\n',
       'How does it feel to date Ivanka Trump?\n',
       'What does salesforce mean?\n'], dtype='<U1170')

In [74]:
find_nearest(query="Why don't i ask a question myself?", k=10)

array(["Why don't I get a date?\n",
       "Why do you always answer a question with a question? I don't, or do I?\n",
       "Why can't I ask a question anonymously?\n",
       "Why don't I get a girlfriend?\n",
       "Why don't I have a boyfriend?\n", "I don't have no question?\n",
       "Why can't I take a joke?\n", "Why don't I ever get a girl?\n",
       "Can I ask a girl out that I don't know?\n",
       "Why don't I have a girlfriend?\n"], dtype='<U1170')

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.